### Train a Policy Network

This notebook trains a policy network, which can predict next move from current features.

In [1]:
import glob
import os
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Activation, Conv2D, Dense, Dropout, Flatten, Input, MaxPooling2D
from tensorflow.keras.models import load_model

from visualization import SvgGoBoard
from train_utils import ParseForPolicy, ParseForPolicy, CreateDataset

tf.enable_eager_execution()

In [2]:
BOARD_SIZE = 19
NUM_CHANNELS = 7
BATCH_SIZE = 64
SHUFFLE_BUFFER = 12800


def BuildModel():
    input = Input(shape=(BOARD_SIZE, BOARD_SIZE, NUM_CHANNELS), name="go_input")
    x = Conv2D(filters=64, kernel_size=(7,7), activation="relu", padding="same",
               data_format='channels_last')(input)
    x = Conv2D(filters=64, kernel_size=(7,7), activation="relu", padding="same",
               data_format='channels_last')(x)
    x = Conv2D(filters=32, kernel_size=(5,5), activation="relu", padding="same",
               data_format='channels_last')(x)
    x = Conv2D(filters=32, kernel_size=(5,5), activation="relu", padding="same",
               data_format='channels_last')(x)
    x = Dropout(0.5)(x)
    x = Conv2D(filters=32, kernel_size=(5,5), activation="relu", padding="same",
               data_format='channels_last')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(512)(x)
    x = Dense(BOARD_SIZE*BOARD_SIZE)(x)
    policy_output = Activation("softmax", name="policy_output")(x)

    model = keras.models.Model(inputs=input, outputs=[policy_output])
    return model

In [3]:
def CompileModel(model):
    model.compile(optimizer="sgd", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    print(model.summary())
    return model

In [ ]:
model = BuildModel()
model = CompileModel(model)
model.summary()

In [5]:
train_set = CreateDataset(glob.glob("/home/tc/SGF/mini/rio/training/data-0000.rio"), BATCH_SIZE)
train_set = train_set.shuffle(buffer_size=SHUFFLE_BUFFER)

In [6]:
model.fit(train_set, epochs=10, steps_per_epoch=1000)

Epoch 1/10
1000/1000 [==============================] - 93s 93ms/step - loss: 5.8797 - acc: 0.0030
Epoch 2/10
1000/1000 [==============================] - 20s 20ms/step - loss: 5.8392 - acc: 0.0034
Epoch 3/10
1000/1000 [==============================] - 20s 20ms/step - loss: 5.7624 - acc: 0.0055
Epoch 4/10
1000/1000 [==============================] - 20s 20ms/step - loss: 5.7223 - acc: 0.0055
Epoch 5/10
1000/1000 [==============================] - 20s 20ms/step - loss: 5.6952 - acc: 0.0065
Epoch 6/10
1000/1000 [==============================] - 20s 20ms/step - loss: 5.6627 - acc: 0.0067
Epoch 7/10
1000/1000 [==============================] - 20s 20ms/step - loss: 5.6363 - acc: 0.0075
Epoch 8/10
1000/1000 [==============================] - 20s 20ms/step - loss: 5.6339 - acc: 0.0083
Epoch 9/10
1000/1000 [==============================] - 20s 20ms/step - loss: 5.6208 - acc: 0.0080
Epoch 10/10
1000/1000 [==============================] - 20s 20ms/step - loss: 5.5926 - acc: 0.0088


In [9]:
test_set = CreateDataset(["/home/tc/SGF/mini/rio/test/data-0000.rio"], BATCH_SIZE)
loss, acc = model.evaluate(test_set, steps=1000)
print("test loss: %f" % loss)
print("test accuracy: %f" % acc)

1000/1000 [==============================] - 12s 12ms/step - loss: 5.5862 - acc: 0.0087
test loss: 5.586160
test accuracy: 0.008703
